주의 사항
1) selenium driver 업데이트 권장드립니다 -> !pip install selenium --upgrade  
2) 크롬창 우측 상단 점 3개 클릭 - 도움말 - chrome 정보에서 chrome 버전 확인하시고 거기에 맞는 웹드라이버 사용 부탁드립니다.  
3) 제가 직접 올린 웹드라이버는 1.16버전으로, 최신 1.15 크롬에서 원할하게 돌아감을 확인했습니다.  

pip install selenium  
pip install request  
pip install webdriver_manager  
등등 필요한 라이브러리 설치 후 실행 요망

In [10]:
import pandas as pd
import numpy as np
import random
import time
import json

from bs4 import BeautifulSoup as bs
import requests

import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from concurrent.futures import ThreadPoolExecutor, as_completed

## 웹드라이버 옵션 추가: 동적 크롤링 성능 향상을 위해 쓸데없는 옵션들 모두 제외
service = Service()
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "none"

def query(game):
    if game == 0:
        print("리그 오브 레전드 인벤의 오늘의 화제글을 불러옵니다.")
        print("-------------------------------------------------------")
        url = "https://www.inven.co.kr/board/lol/4625?my=chuchu"
        
    elif game == 1:
        print("메이플스토리 인벤의 30추글 목록을 불러옵니다.")
        print("-------------------------------------------------------")
        url = "https://www.inven.co.kr/board/maple/5974?my=chuchu"
        
    elif game == 2:
        print("피파온라인 4 인벤의 오늘의 화제글을 불러옵니다.")
        print("-------------------------------------------------------")
        url = "https://www.inven.co.kr/board/fifaonline4/3146?my=chu"
        
    elif game == 3:
        print("디아블로 4 인벤의 30추글 목록을 불러옵니다.")
        print("-------------------------------------------------------")
        url = "https://www.inven.co.kr/board/diablo4/6025?my=chuchu"
        
    elif game == 4:
        print("로스트아크 인벤의 30추글 목록을 불러옵니다.")
        print("-------------------------------------------------------")
        url = "https://www.inven.co.kr/board/lostark/4811?my=chuchu"

    comment_list = []
    
    ## 정적 크롤링으로 화제글 위에서부터 10개 뽑아오기
    html_table = requests.get(url).text
    soup_table = bs(html_table,'html.parser')
    data_box = soup_table.find_all('a',attrs = {'class':'subject-link'})[5:10]
    
    ## 드라이버 생성
    driver = webdriver.Chrome(service=service, options=options)

    for index, data in enumerate(data_box):
        ## 데이터박스 아이템 별로 링크 뽑아서 열기
        link = data.attrs['href']
        driver.get(link)

        try:
            ## 코멘트 래퍼 로딩까지 기다리기
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((
                By.XPATH,'//*[@id="powerbbsCmt2"]/div[2]')))
                
        except TimeoutException:
            print("요소가 나타나지 않았습니다.")
            
        else:
            ## Xpath 형식 동일한 댓글들 모조리 긁어오기
            comment_box = driver.find_elements(
                By.XPATH,'//*[starts-with(@id, "cmt")]/div[2]/div[2]/span')
            
            ## 전체 댓글에서 3개 랜덤추출 후 결과 리스트에 추가
            selected_comments = random.sample(comment_box, 3)
            comment_list = comment_list + [comment.text for comment in selected_comments]
    
    ## 드라이버 닫기
    driver.close()
                        
    print("댓글 추출이 완료되었습니다!")
    print("-------------------------------------------------------")
        
    df = pd.DataFrame(comment_list, columns=['comments'])
    df['comments'] = df['comments'].str.replace('\n',' ')
    df[df['comments']==' ']="이미지만 있는 댓글입니다."
    return df

In [11]:
df_result = pd.DataFrame()
df_result["leagueoflegends"] = query(0)
df_result["maplestory"] = query(1)
df_result["fifa4"] = query(2)
df_result["diablo4"] = query(3)
df_result["lostark"] = query(4)

리그 오브 레전드 인벤의 오늘의 화제글을 불러옵니다.
-------------------------------------------------------
댓글 추출이 완료되었습니다!
-------------------------------------------------------
메이플스토리 인벤의 30추글 목록을 불러옵니다.
-------------------------------------------------------
댓글 추출이 완료되었습니다!
-------------------------------------------------------
피파온라인 4 인벤의 오늘의 화제글을 불러옵니다.
-------------------------------------------------------
댓글 추출이 완료되었습니다!
-------------------------------------------------------
디아블로 4 인벤의 30추글 목록을 불러옵니다.
-------------------------------------------------------
댓글 추출이 완료되었습니다!
-------------------------------------------------------
로스트아크 인벤의 30추글 목록을 불러옵니다.
-------------------------------------------------------
댓글 추출이 완료되었습니다!
-------------------------------------------------------


In [12]:
df_result.head()

,leagueoflegends,maplestory,fifa4,diablo4,lostark
0,좀 치네ㅋㅋㅋㅋㅋㅋㅋ,그럼 사냥터 개인화에 다인사냥터 활성화가 될 수록 매크로가 엿먹는거네?,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...,설마 윙크한거??,스윗남 ㅋㅋㅋ
1,자매품 : 삼대떡,미니던전 삭제하던 짬 남아있네ㅋㅋ,홍박사님을 아세요,ㄹㅇㅋㅋ,ㅋㅋㅋㅋㅋ
2,윗 두명다 병신인건 알겠네,거탐걸면 발작하더라,텐하흐아님? ㅋㅋ,쟤들 본 뒤론 '게임 안해본 새키들이 디4를 논하냐' or '제대로 키워보지도 않고...,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
3,추천 올려서 많은 분들이 보게 해줍시다,19 입니다. 어려운 희망이에요. 너무 슬퍼마세요...,골반이 별론뎅,으 똥망겜 하나하나 나열하니까 진짜 개역겹네,바드는 메로엣타
4,방구석에서 게임만 하는 찌끄래기 인생이라 그저 남 욕하는 게 제일 쉬우니까 그거밖에...,오늘 좋은 운은 아니네요. 주사위는 33 입니다...,ㄷㄷ,맞는말인데 지랄 발광 떠네 병신들 ㅋㅋㅋ,구름 포켓몬입니당...


In [30]:
df_result.to_json('C:/Users/wjdgh/Desktop/comments.json',force_ascii=False, orient = 'columns', indent=4)